## download dataset from http://yann.lecun.com/exdb/mnist/

In [33]:
'''
import requests
import os
from tqdm import tqdm  # for progress bar

def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
    
    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    
    if total_size != 0 and progress_bar.n != total_size:
        print("Error: Failed to download the file.")
        os.remove(destination)  # Remove partially downloaded file
    else:
        print("Download complete.")

# URLs for the MNIST dataset files
base_url = "http://yann.lecun.com/exdb/mnist/"
files = [
    "train-images-idx3-ubyte.gz",
    "train-labels-idx1-ubyte.gz",
    "t10k-images-idx3-ubyte.gz",
    "t10k-labels-idx1-ubyte.gz"
]

# Destination directory for downloaded files
destination_dir = "Downloads"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Download each file
for file in files:
    file_url = base_url + file
    file_destination = os.path.join(destination_dir, file)
    download_file(file_url, file_destination)
'''


'\nimport requests\nimport os\nfrom tqdm import tqdm  # for progress bar\n\ndef download_file(url, destination):\n    response = requests.get(url, stream=True)\n    total_size = int(response.headers.get(\'content-length\', 0))\n    block_size = 1024  # 1 KB\n    progress_bar = tqdm(total=total_size, unit=\'iB\', unit_scale=True)\n    \n    with open(destination, \'wb\') as file:\n        for data in response.iter_content(block_size):\n            progress_bar.update(len(data))\n            file.write(data)\n    progress_bar.close()\n    \n    if total_size != 0 and progress_bar.n != total_size:\n        print("Error: Failed to download the file.")\n        os.remove(destination)  # Remove partially downloaded file\n    else:\n        print("Download complete.")\n\n# URLs for the MNIST dataset files\nbase_url = "http://yann.lecun.com/exdb/mnist/"\nfiles = [\n    "train-images-idx3-ubyte.gz",\n    "train-labels-idx1-ubyte.gz",\n    "t10k-images-idx3-ubyte.gz",\n    "t10k-labels-idx1-ubyt

In [34]:
import tensorflow as tf 


In [35]:
tf.compat.v1.disable_eager_execution()


In [38]:
#tf.compat.v1.flags.DEFINE_integer("is_train", 1, "indicate whether to train the model or use data to predict")
#FLAGS = tf.compat.v1.flags.FLAGS

In [37]:
import tensorflow as tf
import numpy as np
import gzip



tf.compat.v1.disable_eager_execution()

def read_idx_file(file_path):
    with gzip.open(file_path, 'rb') as f:
        # Read the magic number
        magic = int.from_bytes(f.read(4), 'big')
        
        # Read the number of images
        num_images = int.from_bytes(f.read(4), 'big')
        
        # Read the number of rows and columns per image
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')
        
        # Read the image data
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        image_data = image_data.reshape(num_images, rows * cols)  # Flatten images to 1D array
        
        return image_data

def read_idx_labels(file_path):
    with gzip.open(file_path, 'rb') as f:
        # Read the magic number
        magic = int.from_bytes(f.read(4), 'big')
        
        # Read the number of labels
        num_labels = int.from_bytes(f.read(4), 'big')
        
        # Read the label data
        label_data = np.frombuffer(f.read(), dtype=np.uint8)
        
        return label_data

def full_connected_nn(train_images, train_labels, test_images, test_labels):
    
    with tf.compat.v1.variable_scope("data"):
        
        X = tf.compat.v1.placeholder(tf.float32, [None, 784], name="feature")
        y_true = tf.compat.v1.placeholder(tf.float32, [None, 10], name="label")

    with tf.compat.v1.variable_scope("hidden"):
        
        # the first layer randomization for the initialization of parameters 
        weight_hidden = tf.Variable(tf.random.normal([784, 64], mean=0.0, stddev=1.0), name="weightes_1")
        bias_hidden = tf.Variable(tf.random.normal([64], mean=0.0, stddev=1.0), name="biases_1")
        
        a1 = tf.matmul(X, weight_hidden) + bias_hidden
        
    with tf.compat.v1.variable_scope("fc"):
        # the second layer randomization for the initialization of parameters 
        weight_fc = tf.Variable(tf.random.normal([64, 10], mean=0.0, stddev=1.0), name="weightes_2")
        bias_fc = tf.Variable(tf.random.normal([10], mean=0.0, stddev=1.0), name="biases_2")
        
        # fully connected operations 
        # predict ten labels 
        y_predict = tf.matmul(a1, weight_fc) + bias_fc

    with tf.compat.v1.variable_scope("softmax_cross"):
        
        # compute the loss, cross entropy 
        all_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict, name="compute_loss")
        
        # compute the average 
        loss = tf.reduce_mean(all_loss)

        
    with tf.compat.v1.variable_scope("GD"):
        
        # gradient decent 
        train_op = tf.compat.v1.train.GradientDescentOptimizer(0.1).minimize(loss)

        
    with tf.compat.v1.variable_scope("accuracy"):
        
        equal_list = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_predict, 1))
        
        accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))

    tf.summary.scalar("loss", loss)
    tf.summary.scalar("acc", accuracy)

    tf.summary.histogram("weight_hidden", weight_hidden)
    tf.summary.histogram("bias_hidden", bias_hidden)
    tf.summary.histogram("weight_fc", weight_fc)
    tf.summary.histogram("bias_fc", bias_fc)
    
    # merge the results 
    merged = tf.compat.v1.summary.merge_all()
    saver = tf.compat.v1.train.Saver()

    with tf.compat.v1.Session() as sess:
        
        sess.run(tf.compat.v1.global_variables_initializer())
        
        
        
        # get the local model 
        ckpt = tf.compat.v1.train.latest_checkpoint("./tmp/model/")
        
        # check if the model exists 
        if ckpt:
            saver.restore(sess, ckpt)

        if FLAGS['is_train'] == 1:
            
            for i in range(2000):
                
                # create events files 
                file_writer = tf.compat.v1.summary.FileWriter("./tmp/summary/", graph=sess.graph)
                
                # batch size of 50
                mnist_x, mnist_y = train_images.next_batch(50)
                
                mnist_x = train_images[i*50:i*50+50]
                mnist_y = train_labels[i*50:i*50+50]
                
                _, loss_run, acc_run, summary = sess.run([train_op, loss, accuracy, merged], 
                                                          feed_dict={x: mnist_x, y_true: mnist_y})
                
                print(" %d the iteration and the 50 sample loss is %f , accuracy is %f " % (i, loss_run, acc_run))
                
                file_writer.add_summary(summary, i)

                if i % 100 == 0:
                    saver.save(sess, "./tmp/model/fc_nn_model")
                    
        else:
            
            for i in range(100):
                # use 100 test sample to predict 
                
                #image, label = test_images.next_batch(1)
                image = test_images[i:i+1]
                label = test_labels[i:i+1]
                
                print(" the d% sample, real pic number is %d, neural network predicted number is %d" % (
                    i+1,
                    tf.argmax(sess.run(y_true, feed_dict={X: image, y_true: label}), 1).eval(),
                    tf.argmax(sess.run(y_predict, feed_dict={X: image, y_true: label}), 1).eval()
                ))


# Assuming you have the MNIST dataset files in the specified paths
train_images = read_idx_file("Downloads/train-images-idx3-ubyte.gz")
train_labels = read_idx_labels("Downloads/train-labels-idx1-ubyte.gz")

test_images = read_idx_file("Downloads/t10k-images-idx3-ubyte.gz")
test_labels = read_idx_labels("Downloads/t10k-labels-idx1-ubyte.gz")

# Preprocess the data (normalize pixel values)
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert labels to one-hot encoding
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, 10)

# Run the neural network
full_connected_nn(train_images, train_labels_one_hot, test_images, test_labels_one_hot)

 the d1ample, real pic number is 7, neural network predicted number is 0
 the d2ample, real pic number is 2, neural network predicted number is 5
 the d3ample, real pic number is 1, neural network predicted number is 5
 the d4ample, real pic number is 0, neural network predicted number is 8
 the d5ample, real pic number is 4, neural network predicted number is 0
 the d6ample, real pic number is 1, neural network predicted number is 5
 the d7ample, real pic number is 4, neural network predicted number is 5
 the d8ample, real pic number is 9, neural network predicted number is 7
 the d9ample, real pic number is 5, neural network predicted number is 5
 the d10ample, real pic number is 9, neural network predicted number is 0
 the d11ample, real pic number is 0, neural network predicted number is 2
 the d12ample, real pic number is 6, neural network predicted number is 5
 the d13ample, real pic number is 9, neural network predicted number is 7
 the d14ample, real pic number is 0, neural net